## Vision Transformer (ViT) 预训练模型微调配置

本模块配置了一个基于 **ViT-B/16 (Base/16)** 预训练权重的深度学习模型，并采用**分层微调策略**以适应新的分类任务。

### 策略说明

1.  **加载模型：** 使用 PyTorch `torchvision.models` 加载 ViT-B/16 预训练模型。
2.  **参数冻结：** 默认**冻结所有** ViT 编码器层的参数，以保持预训练获得的通用特征知识（特征提取）。
3.  **解冻高层：** 仅**解冻最后一个 Transformer Block**（索引 11），允许模型对最高级特征进行微调，以适应特定任务。
4.  **替换分类头：** 移除了原有的分类头，替换为一个包含 **ReLU 激活**和 **Dropout (0.4)** 的新 **`nn.Sequential`** 分类器，其输出维度匹配目标类别数（`num_classes`）。

### 参数概览

| 统计项 | 数值 | 目的 |
| :--- | :--- | :--- |
| 模型总参数量 | 约 87.17 M | ViT-B/16 的标准参数量。 |
| **可训练参数量** | **约 7.69 M** | 仅包括最后一个 Block 和新分类头，显著减少了训练负担。 |
| 分类头参数量 | 约 0.60 M | 新添加的、需要从头学习的参数。 |

注：模型需要的输入大小为 Image_size = (224,224)

In [ ]:
import torch
from torch import nn
from torchvision.models import vit_b_16, ViT_B_16_Weights

# 参数设置
num_classes = 10 # 分类数量
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 使用效果最好的 ViT-B/16 预训练权重版本
model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
# 冻结所有参数 (特征提取阶段)
for param in model.parameters():
    param.requires_grad = False
# 解冻最后一个 Block (索引 11) 的参数
for param in model.encoder.layers[-1].parameters():
    param.requires_grad = True
print("注意：已解冻最后一个 Transformer 编码器块的参数。")

# 获取 ViT 分类头 (model.head) 的输入特征数
# 对于 ViT-B/16，这个值是 768
num_ftrs = 768

# 修改分类头 (model.head) 以适应你的分类任务
# ViT 的分类头就是模型在 `cls` token 上接的一个全连接层
model.head = nn.Sequential(
    # 第一层从 ViT 的输出 (768) 开始
    nn.Linear(num_ftrs, 768),  # 注意：这里我们使用 2048 来匹配你 ResNet 示例中的结构
    nn.ReLU(),
    nn.Dropout(0.4),
    
    # 输出层：连接到你的类别数
    nn.Linear(768, num_classes)
)

# 将模型移动到GPU（如果可用）
model.to(device)

# 打印模型结构
print(model)

# 打印可训练的参数数量 (用于确认冻结策略是否生效)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n模型总参数量: {total_params / 1e6:.2f} M")
print(f"可训练参数量: {trainable_params / 1e6:.2f} M")
print(f"微调层参数量 (model.head): {sum(p.numel() for p in model.head.parameters()) / 1e6:.2f} M")

## ResNet50 预训练模型微调配置

本模块配置了一个基于 **ResNet50** 预训练权重的深度学习模型，采用**分层微调 (Partial Fine-Tuning)** 策略，以高效适应新的 $\mathbf{N=10}$ 类别分类任务。

### 策略说明

1.  **加载模型：** 使用 PyTorch `torchvision.models` 加载 ResNet50 预训练权重。
2.  **参数冻结：** 默认**冻结大部分参数**（包括 `layer1`、`layer2`、`layer3`），保留浅层和中层预训练获得的通用特征。
3.  **解冻高层：** 仅**解冻最后一个卷积块 (`model.layer4`)** 的参数，允许模型对最高级特征进行微调。
4.  **替换分类头：** 移除了原有的全连接层 (`model.fc`)，替换为一个包含两层线性层、ReLU 激活和 $\mathbf{40\%}$ Dropout 的新分类器结构。输入特征数固定为 $\mathbf{2048}$。

### 参数概览 (基于代码输出)

| 统计项 | 描述 |
| :--- | :--- |
| 模型总参数量 | 约 25.56 M |
| **可训练参数量** | 仅包括 `layer4` 和新的分类头 (`model.fc`) |
| **微调层参数量** | 仅新分类头 (`model.fc`) 的参数量 |

**核心：** 这种策略（冻结大部分 + 微调高层和分类头）旨在利用预训练的鲁棒特征，同时最大限度地减少训练负担和过拟合的风险。

注：模型需要的输入大小为 Image_size = (224,224)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# 前置参数
num_classes = 10 # 类别数
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练的效果最好的resnet50权重版本
model = resnet50(weights=ResNet50_Weights.DEFAULT)

# 冻结所有参数
for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True

# 获取 ResNet50 最后一个全连接层 (model.fc) 的输入特征数
# 对于 ResNet50，这个值固定是 2048
num_ftrs = model.fc.in_features 

# 修改分类层 (model.fc) 以适应你的分类任务
# 这里使用你提供的更复杂的分类器结构作为示例
model.fc = nn.Sequential(
    # 第一层从 ResNet50 的输出 (2048) 开始
    nn.Linear(num_ftrs, 2048),
    nn.ReLU(),
    nn.Dropout(0.4),
    
    # 输出层：连接到你的类别数
    nn.Linear(2048 , num_classes)
)

# 将模型移动到GPU（如果可用）
model.to(device)

# 打印模型结构
print(model)

# 打印可训练的参数数量 (用于确认冻结策略是否生效)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n模型总参数量: {total_params / 1e6:.2f} M")
print(f"可训练参数量: {trainable_params / 1e6:.2f} M")
print(f"微调层参数量 (model.head): {sum(p.numel() for p in model.head.parameters()) / 1e6:.2f} M")